# 기본적 데이터 전처리

In [2]:
import pandas as pd
from pandas import Series
import numpy as np

from google.colab import files 
uploaded = files.upload()

Saving vgames2.csv to vgames2 (1).csv


In [3]:
df = pd.read_csv("vgames2.csv", index_col = 0)
df

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0.00,0.00,0.00
2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0.00,0.00,0.01
3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0.00,0.00,0.02,0.00
4,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0.00,0.00,0.00
5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0.00,0.04
...,...,...,...,...,...,...,...,...,...
16594,Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,0.15,0.04,0.00,0.01
16595,Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,0.01,0.00,0.00,0.00
16596,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13
16597,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,0.05,0.05,0.25,0.03


데이터 전처리를 어느 정도 마친 엑셀 파일을 불러오고, Unnamed :0를 제거해주었습니다.

저희가 년도별 트렌드 분석을 해야하는 부분이 있어 Year 부분에 있는 결측치와 고르지 못한 자료, 그리고 잘못된 자료들을 일일이 수작업으로 검증하며 고쳤습니다.
이는 데이터 자체가 잘못된 것들이라 직접 엑셀에 손을 대는 수밖에 없었습니다.

**사족으로, 6393번의 Brothers in Arms: Furious 4 는 개발 과정에서 문제가 생겨 아예 개발이 취소된 작품인데 출고량이 잡혀있습니다. 혹시 개발 중에 예약판매나 클라우드 펀딩을 감수했나 조사해봤는데 그것 역시 해당되지 않는 케이스였습니다. 데이터를 제거할까 고민했으나 너무 뒤늦게 알게 된 케이스라 그냥 그대로 진행했습니다ㅠ**

In [4]:
df["NA_Sales"] = 100 * df["NA_Sales"]

In [5]:
df["EU_Sales"] = 100 * df["EU_Sales"]

In [6]:
df["JP_Sales"] = 100 * df["JP_Sales"]

In [7]:
df["Other_Sales"] = 100 * df["Other_Sales"]

In [8]:
df

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,4.0,0.0,0.0,0.0
2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,17.0,0.0,0.0,1.0
3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0.0,0.0,2.0,0.0
4,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,4.0,0.0,0.0,0.0
5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,12.0,9.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...
16594,Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,15.0,4.0,0.0,1.0
16595,Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,1.0,0.0,0.0,0.0
16596,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,44.0,19.0,3.0,13.0
16597,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,5.0,5.0,25.0,3.0


수치를 쉽게 숫자로 받아들일 수 있게 전체 column에 곱셈을 해주었습니다. 단위는 만입니다.

이렇게 기본적인 데이터의 전처리를 마쳤습니다. 

# 추가적으로 공부해서 메꿔야 할 부분
1. *M,K 등의 숫자 밑 단위를 코딩으로 쉽게 처리가 가능하다. 이를 배워보도록하자(물론 실패한 것이지만...)*
2. *각 column에 100을 곱하는 과정을 굳이 코딩 식을 4개씩이나 써서 할 필요가 없을 것 같다(물론 이것 역시도 실패한 것이지만...)* 

# 1번. 지역에 따라서 선호하는 장르가 다를까?

이를 입증하기 위해 먼저 특정 장르를 선별하여 따로 분리해낸 이후 그 장르가 다른 지역에서만 높은 출고량을 갖는지, 즉 특별하게 선호되는 장르가 있는지 확인해보도록 하겠습니다.

다만 이를 실행하기 이전에 먼저 구해야하는 값들이 있습니다.
### 바로 **인구 수 대비 구매력**입니다. 
*(이 때 Other_Sales는 정확한 나라에 대한 정보가 없으므로 계산하지 않도록 하겠습니다)*

NA이라는 지역으로 묶인 곳의 인구수는 JP와 비교하면 큰 차이를 보일 수 밖에 없습니다. 이는, NA에서 출고량이 높다고 해서 JP보다 무조건적으로 선호도가 높지 않다는 의미가 됩니다.

이 때문에 각 지역별로
"평균적인 출고량"이 **인구수 대비 어느 정도인지 먼저 알아본 이후에** 
과연 구한 값이 이레귤러인지 아닌지 자료 분석할 예정입니다.

In [9]:
df["NA_Sales"].sum()

439295.0

In [10]:
df["EU_Sales"].sum()

243413.0

In [11]:
df["JP_Sales"].sum()

129102.0

지역별로 전체 출고량을 합쳤습니다.
이후 각 지역의 인구로 전체 출고량을 나누어서 평균을 구할 생각입니다.

인구수는 대략적으로 인터넷에서 검색해본 결과 일본 1.2억, 미국 3.2억, 유럽 7.4억이 나왔습니다.

In [12]:
NAM = 439295 / 3.2
EUM = 243425 / 7.4
JPM = 129090 / 1.2

print(NAM, EUM, JPM)

137279.6875 32895.270270270266 107575.0


In [13]:
D1 = NAM / EUM
D2 = JPM / EUM
print(D1, D2)

4.173234825921742 3.2702269692923904


평균 출고량을 구한 이후 (NAM, EUM, JPM - 지역별 Mean)
인구수가 가장 많았던 유럽의 결과값으로 나누어서 NA와 JP의 구매력이 몇 배가 되는지 구해봤습니다.

미국의 경우 대략 4.2배, 일본의 경우 대략 3.3배가 되는 것을 확인했습니다(반올림)

# 인구 대비 평균 출고량

# 1(EU) : 4.2(NA) : 3.3(JP)

*이렇게 보면 인구 대비 유럽의 출고량이 너무 낮게 잡혀 이상하다고 생각하실 수도 있는데, 현실을 고려하면 이것이 맞는 수치입니다.*

*그 이유는 일단 첫째로, 유럽으로 묶인 지역이 너무 광범위하게 넓습니다. 때문에 인구수가 많을 수 밖에 없는 환경입니다. 그런데 그 인구수에 맞지 않게 게임 출고량, 즉 구매량이 적은 이유는 바로 문화와 환경적 차이 때문입니다.*


*대도시를 중심으로 삼는 일본과 미국과는 달리(미국은 대도시가 많은 만큼 시골 지역도 많긴 합니다만)*

*각 나라 별로 경제적인 발전과 인터넷의 발전이 미국이나 일본에 비해 현저하게 떨어지는 지역이 많고, 이에 따라 게임을 사서 즐기는 사람의 숫자가 굉장히 큰 차이를 보입니다.*

그 외에도 여러 문화적인 차이가 있기 때문에 이와 같은 결과가 일어나는 것은 당연하다고 볼 수가 있습니다.

이제 특정 장르를 뽑아서 과연 위에서 구한 값과 얼마나 차이가 나는지, 즉 선호도에 따라 차이가 날 것인지 아닌지를 알아볼 것입니다.

In [14]:
is_sracing = df["Genre"] == "Racing"
sracing = df[is_sracing]
sracing

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
11,Ford Racing,PS,2001.0,Racing,Empire Interactive,48.0,33.0,0.0,6.0
27,Starwinder: The Ultimate Space Race,PS,1996.0,Racing,Mindscape,3.0,2.0,0.0,0.0
30,McGrath Vs. Pastrana Freestyle Motocross,PS,2000.0,Racing,Acclaim Entertainment,14.0,10.0,0.0,2.0
38,Beamrider,2600,1984.0,Racing,Activision,26.0,1.0,0.0,0.0
50,Dreamworks Madagascar Kartz,X360,2009.0,Racing,Activision,14.0,9.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...
16519,R: Racing Evolution,GC,2003.0,Racing,Namco Bandai Games,6.0,2.0,0.0,0.0
16534,Mega Man: Battle & Chase,PS,1997.0,Racing,Capcom,0.0,0.0,6.0,0.0
16540,Juiced 2: Hot Import Nights,DS,2007.0,Racing,THQ,22.0,0.0,0.0,2.0
16543,Richard Burns Rally,PC,2004.0,Racing,SCi,0.0,3.0,0.0,1.0


Racing 장르만 따로 빼냈습니다. 이후 평균치를 구하는 방식은 위와 동일하게 진행할 예정입니다.

그 이전에 먼저 각 지역별로 총 출고량 먼저 살펴보겠습니다.

In [15]:
rm1 = sracing["NA_Sales"].sum()
rm2 = sracing["EU_Sales"].sum()
rm3 = sracing["JP_Sales"].sum()

print(rm1, rm2, rm3)

35719.0 23710.0 5640.0


이제 각 지역별 출고량에 인구수를 나누어서 위에서 구했던 평균적인 구매력 대비 수치 차이가 어느 정도 나는지 알아보도록 하겠습니다.

In [16]:
rm1 = rm1 / 3.2
rm2 = rm2 / 7.4
rm3 = rm3 / 1.2

r1 = rm1 / rm2
r2 = rm3 / rm2

print(r1, r2)

3.4837700337410378 1.466891606916913


그러나 일반적으로 보였던 개인당 구매량

EU가 보인 Racing 장르의 출고량이 1일 시 NA는 6.6으로 위에서 봤던 평균 출고량, 4.2와 굉장히 큰 차이를 보입니다.

JP는 정반대로 1.4로, 위에서 봤던 평균 출고량 3.3에 비해서 현저하게 떨어진 수치를 보입니다.

이를 통해 저희는 Racing 종목이 NA 지역에서 굉장히 선호되고 있으며, 일본에서는 상대적으로 그렇지 않다는 것을 알 수가 있습니다.

In [17]:
is_role = df["Genre"] == "Role-Playing"
srole = df[is_role]
srole

rom1 = srole["NA_Sales"].sum()
rom2 = srole["EU_Sales"].sum()
rom3 = srole["JP_Sales"].sum()

rom1 = rom1 / 3.2
rom2 = rom2 / 7.4
rom3 = rom3 / 1.2

ro1 = rom1 / rom2
ro2 = rom3 / rom2

print(ro1, ro2)

4.0251457130406365 11.58727348658169


다른 예시로 이번엔 Role-Playing 게임을 들고 왔습니다.

NA에서는 4.025 정도로 기존의 4.2와 거의 비슷한 값이 나왔습니다.

그러나 반대로 일본에서는 11.59라는 무시무시한 수치를 보이면서 기존의 3.3과는 비교도 안 될 정도로 큰 강세를 보여주고 있습니다. 거의 3배에 달하는 이 수치는 JP에서 Role-Playing 장르에 대해 강한 선호도를 갖고 있다는 증거로 충분하다고 생각됩니다.

# 2. 연도별 게임의 트렌드가 있을까
위의 가설을 검증하기 위해 이번에도 역시 먼저 짚고 넘어가야할 부분이 있습니다.

### **그것은 바로 게임산업이 가지고 있는 한계점이 있다는 것입니다.**
이는 자료와 함께 밑에서 더 설명하도록 하겠습니다.

**### 또 다른 문제는 바로 "트렌드가 무엇인가"**
라는 문제입니다.

특정 게임이 유행하고 나면 비슷한 게임이 뜨는 현상이 생깁니다(어몽어스 이후 마피아 게임)
반대로 특정 게임이 유행하고 나면 비슷한 게임이 쏟아지는 현상도 있습니다(개발자 입장)

첫 번째는 소비자들의 트렌드입니다
당시 어떤 게임이 재밌음을 느끼고 비슷한 류의 게임을 찾는 노력이 있고, 그게 트렌드를 만든다는 것
근데 여기에서 중요한 점이 무엇이냐하면 이것은 출고량에 “그대로 반영되는 것이 아니라는 것"입니다.
오히려 전 분기, 전 년도에 낮았던 출고량이 늘어나는 방식 등을 통해 파악이 가능한데, 주어진 것은 총 전체적인 출고량이기 때문에 제대로 알 수가 없습니다.

두 번째는 게임을 만드는 개발자로서의 트렌드입니다.
콘솔, 컴퓨터 게임의 시장은 모바일과는 달리 개발에 시간이 오래 걸리고, 환경적인 요소에 영향을 많이 받습니다.
저희에게 주어진 데이터는 콘솔 및 컴퓨터 게임의 시장인데, 특정 장르가 흥한다고 해서 바로 그 다음 분기, 혹은 그 다음 년도에 작품들이 쏟아져 나오는 것이 불가능합니다.
개발 기간도 있고, 비슷한 류의 게임이 몰려나오면 오히려 팀킬이 벌어지는 경우도 있다는 점입니다. 그리고, 이 역시 출고량이라는 데이터로는 증가폭을 알 수가 없기 때문에 문제가 된다는 점이 있습니다.
그 외에도 개발 환경이나 여러 요소 등에 따라 여러 문제가 생기기 때문에, 개인적인 생각으로는 주어진 자료인 출고량으로는 질문에 대한 답을 하는 것이 불가능하다고 생각합니다.

그러나 그럼에도 불구하고 저희는 답을 찾아내야 하기에, 이를 찾아낼 방법을 고안해보았습니다.

먼저 트렌드, 즉 유행이 있는지 먼저 점검을 하기 위해 년도별로 정리를 한 이후, 특정 년도에서 특정 장르가 많이 나왔는지를 살펴볼 생각입니다.

In [18]:
year = df.sort_values(by = 'Year', ascending = True)
df1 = year
df1

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
6119,Wheel of Fortune,2600,1975.0,Misc,Unknown,47.0,36.0,0.0,12.0
11317,Air-Sea Battle,2600,1977.0,Shooter,Atari,91.0,6.0,0.0,1.0
13903,Indy 500,2600,1977.0,Racing,Atari,26.0,1.0,0.0,0.0
14766,Combat,2600,1977.0,Action,Atari,117.0,7.0,0.0,1.0
5231,Circus Atari,2600,1977.0,Action,Atari,43.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2796,Call of Duty: Modern Warfare Trilogy,X360,2016.0,Shooter,Activision,1.0,1.0,0.0,0.0
16389,Project Setsuna,PSV,2016.0,Role-Playing,Square Enix,0.0,0.0,5.0,0.0
9677,Mirror's Edge Catalyst,PS4,2016.0,Platform,Electronic Arts,10.0,14.0,2.0,6.0
6393,Brothers in Arms: Furious 4,X360,NaN,Shooter,NaN,1.0,0.0,0.0,0.0


*Brothers in Arms: Furious 4	위에서 잠깐 언급했었는데, E3에서 제작하기로 발표했던 작품이지만 개발과정에서 문제가 생겨서 아예 출시되지 않았습니다.*

*Housekeeping은 아무리 뒤져도 언제 발매된 작품인지 찾을 수가 없어서(사실은 작품 자체를 찾지 못했습니다) 그냥 결측치로 처리했습니다.*

일단 년도별로 데이터프레임 정리를 마쳤습니다.

In [19]:
trend = df1["Genre"].groupby(df1["Year"])
trend.size()

Year
1975.0       1
1977.0       4
1978.0       6
1979.0       1
1980.0      14
1981.0      46
1982.0      38
1983.0      17
1984.0      15
1985.0      14
1986.0      21
1987.0      15
1988.0      15
1989.0      18
1990.0      16
1991.0      41
1992.0      43
1993.0      62
1994.0     121
1995.0     219
1996.0     264
1997.0     292
1998.0     381
1999.0     342
2000.0     353
2001.0     490
2002.0     852
2003.0     785
2004.0     776
2005.0     955
2006.0    1032
2007.0    1218
2008.0    1456
2009.0    1451
2010.0    1284
2011.0    1177
2012.0     664
2013.0     549
2014.0     584
2015.0     615
2016.0     349
Name: Genre, dtype: int64

그룹바이를 진행한 이후 각 년도별로 몇 개의 게임이 나왔는지를 한 번 체크했습니다.

이것을 진행한 이유가 무엇이냐면 1993년도까지의 데이터를 아예 제외시키기 위함인데, 그 이유는 간단합니다. 왜냐면 이때 당시의 게임들의 장르 및 유행을 무시할 수 있는 이유가 충분하기 때문입니다.

그 이유는 바로 위에서 언급했던 게임의 한계 때문입니다. 1990년도까지, 매해 개발된 게임의 갯수는 100개가 되지 않습니다. 그 이유는 바로 "기술력"의 한계 때문입니다. 당시에는 기술력이 발달되지 않아 만들 수 있는 게임의 장르 자체가 제한되어 있었으며, 게임 산업이 막 태동했을 때라 제대로 된 장르 개발이 되지 않았을 때입니다.

이후 90년도 중반 대에 들어서면서 개발되는 게임의 숫자가 기하급수적으로 늘어나는데, 이는 개인 pc와 인터넷 망의 보급이 이루어지면서 가능했던 일입니다.

때문에 저는 93년도 이하의 기록들은 유행과는 큰 상관이 없다는 입장입니다. 게임이라는 것 자체가 유행을 타면 탔지, 특정 장르가 유행을 탄 것이 아니라는 말입니다.

또한, 위의 수치를 보면 2012년도부터 나오는 게임의 숫자가 또 다시 줄어드는데, 이는 2012년에 여러 대작 게임들이 쏟아지며(보더랜드, 파크라이, 저니, 엑스콤, 그 외 등등) 게이머들의 눈이 한 단계 올라갔으며, 크라우드 펀딩이 이때부터 시작되면서부터 퀼리티를 챙기는 대신 게임별 개발 기간이 늘었기 때문입니다.

이에 저는 1994년부터 2011년까지의 데이터를 가지고 당시에 유행이라고 할만한, 트렌드라고 할 만한 흐름이 있었는지 살펴보기로 했습니다.

In [20]:
df1

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
6119,Wheel of Fortune,2600,1975.0,Misc,Unknown,47.0,36.0,0.0,12.0
11317,Air-Sea Battle,2600,1977.0,Shooter,Atari,91.0,6.0,0.0,1.0
13903,Indy 500,2600,1977.0,Racing,Atari,26.0,1.0,0.0,0.0
14766,Combat,2600,1977.0,Action,Atari,117.0,7.0,0.0,1.0
5231,Circus Atari,2600,1977.0,Action,Atari,43.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2796,Call of Duty: Modern Warfare Trilogy,X360,2016.0,Shooter,Activision,1.0,1.0,0.0,0.0
16389,Project Setsuna,PSV,2016.0,Role-Playing,Square Enix,0.0,0.0,5.0,0.0
9677,Mirror's Edge Catalyst,PS4,2016.0,Platform,Electronic Arts,10.0,14.0,2.0,6.0
6393,Brothers in Arms: Furious 4,X360,NaN,Shooter,NaN,1.0,0.0,0.0,0.0


In [21]:
df2 =df1.drop("Name", axis = 1)
df2

,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
6119,2600,1975.0,Misc,Unknown,47.0,36.0,0.0,12.0
11317,2600,1977.0,Shooter,Atari,91.0,6.0,0.0,1.0
13903,2600,1977.0,Racing,Atari,26.0,1.0,0.0,0.0
14766,2600,1977.0,Action,Atari,117.0,7.0,0.0,1.0
5231,2600,1977.0,Action,Atari,43.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2796,X360,2016.0,Shooter,Activision,1.0,1.0,0.0,0.0
16389,PSV,2016.0,Role-Playing,Square Enix,0.0,0.0,5.0,0.0
9677,PS4,2016.0,Platform,Electronic Arts,10.0,14.0,2.0,6.0
6393,X360,NaN,Shooter,NaN,1.0,0.0,0.0,0.0


In [22]:
df2 = df2.drop("Platform", axis = 1)
df2

,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
6119,1975.0,Misc,Unknown,47.0,36.0,0.0,12.0
11317,1977.0,Shooter,Atari,91.0,6.0,0.0,1.0
13903,1977.0,Racing,Atari,26.0,1.0,0.0,0.0
14766,1977.0,Action,Atari,117.0,7.0,0.0,1.0
5231,1977.0,Action,Atari,43.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...
2796,2016.0,Shooter,Activision,1.0,1.0,0.0,0.0
16389,2016.0,Role-Playing,Square Enix,0.0,0.0,5.0,0.0
9677,2016.0,Platform,Electronic Arts,10.0,14.0,2.0,6.0
6393,NaN,Shooter,NaN,1.0,0.0,0.0,0.0


In [23]:
df2 = df2.drop("Publisher", axis = 1)
df2

,Year,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales
6119,1975.0,Misc,47.0,36.0,0.0,12.0
11317,1977.0,Shooter,91.0,6.0,0.0,1.0
13903,1977.0,Racing,26.0,1.0,0.0,0.0
14766,1977.0,Action,117.0,7.0,0.0,1.0
5231,1977.0,Action,43.0,3.0,0.0,0.0
...,...,...,...,...,...,...
2796,2016.0,Shooter,1.0,1.0,0.0,0.0
16389,2016.0,Role-Playing,0.0,0.0,5.0,0.0
9677,2016.0,Platform,10.0,14.0,2.0,6.0
6393,NaN,Shooter,1.0,0.0,0.0,0.0


In [24]:
NAS = df2["NA_Sales"].groupby([df2["Year"],df2["Genre"]]).sum()
NAS

Year    Genre       
1975.0  Misc             47.0
1977.0  Action          160.0
        Racing           26.0
        Shooter          91.0
1978.0  Action           25.0
                        ...  
2016.0  Role-Playing    139.0
        Shooter         749.0
        Simulation        0.0
        Sports          457.0
        Strategy         11.0
Name: NA_Sales, Length: 398, dtype: float64

In [25]:
EUS =  df2["EU_Sales"].groupby([df2["Year"],df2["Genre"]]).sum()

In [26]:
JPS =  df2["JP_Sales"].groupby([df2["Year"],df2["Genre"]]).sum()
OTS =  df2["Other_Sales"].groupby([df2["Year"],df2["Genre"]]).sum()

In [34]:
MG1 = NAS + EUS
MG1

Year    Genre       
1975.0  Misc              83.0
1977.0  Action           170.0
        Racing            27.0
        Shooter           97.0
1978.0  Action            27.0
                         ...  
2016.0  Role-Playing     268.0
        Shooter         1520.0
        Simulation         9.0
        Sports          1193.0
        Strategy          46.0
Length: 398, dtype: float64

In [36]:
MG2 = NAS + EUS + JPS + OTS
MG2

Year    Genre       
1975.0  Misc              95.0
1977.0  Action           171.0
        Racing            27.0
        Shooter           98.0
1978.0  Action            27.0
                         ...  
2016.0  Role-Playing     679.0
        Shooter         1823.0
        Simulation        39.0
        Sports          1463.0
        Strategy          55.0
Length: 398, dtype: float64

In [49]:
df2.merge(df2, "MG2", how = "inner", axis = 1)

TypeError: ignored